<a href="https://colab.research.google.com/github/MicaelaRomeroC3/COPs/blob/main/02_Update_historic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from datetime import datetime

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# ---------------------
# Configuración de rutas
# ---------------------
ruta_clean = '/content/drive/MyDrive/COPS/DATA/2.CLEANED'
ruta_historical = '/content/drive/MyDrive/COPS/DATA/3.HISTORICAL/llamadas.parquet'

# ---------------------
# 1. Detectar el archivo limpio más reciente
# ---------------------
archivos_clean = [f for f in os.listdir(ruta_clean) if f.endswith('.csv')]
if not archivos_clean:
    print("⚠️ No hay archivos limpios en la carpeta CLEANED.")
    exit()

archivo_mas_reciente = max(archivos_clean, key=lambda f: os.path.getmtime(os.path.join(ruta_clean, f)))
ruta_limpio = os.path.join(ruta_clean, archivo_mas_reciente)

print(f"\n📥 Cargando archivo limpio: {archivo_mas_reciente}")
df_nuevo = pd.read_csv(ruta_limpio)

# 2. Cargar histórico si existe
# ---------------------
if os.path.exists(ruta_historical):
    df_hist = pd.read_parquet(ruta_historical)
    print("📚 Histórico cargado con", len(df_hist), "registros.")
else:
    df_hist = pd.DataFrame()
    print("🆕 No se encontró histórico, se creará uno nuevo.")

# 3. Unir y eliminar duplicados
# ---------------------

df_combinado = pd.concat([df_hist, df_nuevo], ignore_index=True)

# 👇 Aquí defines la clave única. Ajusta según tus datos reales.
clave = ['ID']
df_combinado = df_combinado.drop_duplicates(subset=clave)

print(f"🔁 Total después de eliminar duplicados: {len(df_combinado)} registros.")

#Ante descuadre de nota mirar el siguiente código:
for col in df_combinado.columns:
    if df_combinado[col].dtype == object:
        valores_unicos = df_combinado[col].dropna().unique()
        if set(valores_unicos).issubset({'True', 'False', 'No Procede'}):
            df_combinado[col] = df_combinado[col].map({'True': True, 'False': False, 'No Procede': False})
        else:
            # 👇 Si hay mezcla rara, intentamos forzar donde se pueda
            if 'True' in valores_unicos or 'False' in valores_unicos or 'No Procede' in valores_unicos:
                df_combinado[col] = df_combinado[col].apply(lambda x: True if x == 'True' else (False if x == 'False' or x == 'No Procede' else pd.NA))
                df_combinado[col] = df_combinado[col].astype('boolean')

# ---------------------
# 4. Guardar histórico actualizado
# ---------------------
os.makedirs(os.path.dirname(ruta_historical), exist_ok=True)
df_combinado.to_parquet(ruta_historical, index=False)

print(f"\n✅ Histórico actualizado guardado en: {ruta_historical}")


📥 Cargando archivo limpio: llamadas_clean_20251024.csv


/tmp/ipython-input-479304840.py:19: DtypeWarning: Columns (103,104,105,106,111,112) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nuevo = pd.read_csv(ruta_limpio)


📚 Histórico cargado con 0 registros.
🔁 Total después de eliminar duplicados: 10000 registros.

✅ Histórico actualizado guardado en: /content/drive/MyDrive/COPS/DATA/3.HISTORICAL/llamadas.parquet


**-------------------------------------------------------**

**---------------------------------------------------------**

# **Para VACIAR histórico**

In [3]:
import pandas as pd

ruta = '/content/drive/MyDrive/COPS/DATA/3.HISTORICAL/llamadas.parquet'

# Cargar el parquet para conservar las columnas
df = pd.read_parquet(ruta)

# Vaciar el contenido (cero filas)
df_vacio = df.iloc[0:0]

# Sobrescribir el archivo con la estructura vacía
df_vacio.to_parquet(ruta, index=False)

print("✅ Histórico vaciado correctamente. El archivo está en blanco, con 0 registros.")


✅ Histórico vaciado correctamente. El archivo está en blanco, con 0 registros.
